In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import json
import cv2
import sklearn
from tensorflow.keras import regularizers
from tensorflow.keras import layers
import os

In [2]:
GESTURE_TYPES = 11
LABEL_DICT = {k:i for i,k in enumerate([21, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33])}
CONNECTION_LABELS = [
    (0, 1), (1, 2), (2, 3), (3, 4),
    (5, 6), (6, 7), (7, 8),
    (9, 10), (10, 11), (11, 12),
    (13, 14), (14, 15), (15, 16),
    (17, 18), (18, 19), (19, 20),
    (0, 5), (5, 9), (9, 13), (13, 17), (0, 17)
]

In [3]:
def load_keypoint_sequences(data_path='gesture_recognition/Fall 2020/data'):
    keypoints = []
    labels = []
    for subjectName in os.listdir(data_path):
        if not (subjectName.startswith("Subject") or subjectName.startswith("subject")): continue
        # subjectNum = int(re.findall(r'(\d+)', subjectName)[0])
        for sceneName in os.listdir(os.path.join(data_path, subjectName)):
            if not (sceneName.startswith("Scene") or subjectName.startswith("scene")): continue
            for groupEntry in os.scandir(os.path.join(data_path, subjectName, sceneName)):
                with open(groupEntry, 'r') as f:
                    groupData = json.load(f)
                    for gesture in groupData:
                        # print(gesture['label'], gesture['keypoints'])
                        for i in range(len(gesture['keypoints'])):
                            if not gesture['keypoints'][i]:
                                gesture['keypoints'][i] = [[np.nan, np.nan, np.nan] for _ in range(21)]
                        keypoints.append(gesture['keypoints'])
                        labels.append(LABEL_DICT[gesture['label']])
    keypoints = tf.keras.preprocessing.sequence.pad_sequences(keypoints,dtype='float32',padding='post',value=np.NaN)
    labels = np.array(labels)
    return keypoints, labels
keypoints, labels = load_keypoint_sequences()
print(keypoints.shape, labels.shape)

(3141, 37, 21, 3) (3141,)


In [4]:
def generate_connection_angles_from_sequences(keypoints, keypoints_num=21, keypoints_dimensions=3):
    connections = []
    for connection in CONNECTION_LABELS:
        connections.append(keypoints[..., connection[1], :] - keypoints[..., connection[0], :])
    connections = np.stack(connections, axis = -2)
    tensor1 = connections[..., np.newaxis].repeat(keypoints_num, -1).transpose(0,1,2,4,3)
    tensor2 = connections[..., np.newaxis].repeat(keypoints_num, -1).transpose(0,1,4,2,3)
    angles = (tensor1*tensor2).sum(axis=-1)/np.linalg.norm(tensor1,axis=-1)/np.linalg.norm(tensor2,axis=-1)
    angles = angles.transpose(2,3,0,1)[np.triu_indices(21, k = 1)].transpose(1,2,0)
    return np.arccos(angles)
angles = generate_connection_angles_from_sequences(keypoints)
print(angles.shape)

(3141, 37, 210)


<ipython-input-4-c423b6856ef8>:10: RuntimeWarning: invalid value encountered in arccos
  return np.arccos(angles)


In [5]:
def generate_joint_distances_from_sequences(keypoints, keypoints_num=21, keypoints_dimensions=3):
    connections = []
    for connection in CONNECTION_LABELS:
        connections.append(keypoints[..., connection[1], :] - keypoints[..., connection[0], :])
    connections = np.stack(connections, axis = -2)
    tensor1 = connections[..., np.newaxis].repeat(keypoints_num, -1).transpose(0,1,2,4,3)
    tensor2 = connections[..., np.newaxis].repeat(keypoints_num, -1).transpose(0,1,4,2,3)
    distances = np.linalg.norm(tensor1-tensor2,axis=-1).transpose(2,3,0,1)[np.triu_indices(21, k = 1)].transpose(1,2,0)
    return distances
distances = generate_joint_distances_from_sequences(keypoints)
print(distances.shape)

(3141, 37, 210)


In [6]:
def visualize_keypoint_sequences(keypoints):
    for sequence in keypoints:
        for points in sequence:
            img = np.zeros((480, 640, 3))
            for point in points:
                x, y, z = point
                if np.isnan(x):
                    continue
                cv2.circle(img, (int(x), int(y)), 4, (255, 0, 0), 2)
            for connection in CONNECTION_LABELS:
                if np.isnan(points[connection[0]][0]):
                    continue
                x0, y0, z0 = points[connection[0]]
                x1, y1, z1 = points[connection[1]]
                cv2.line(img, (int(x0), int(y0)), (int(x1), int(y1)), (0, 255, 0), 2)
            cv2.imshow("Key Points", img)
            key = cv2.waitKey(1)
            if key == 27:
                cv2.destroyAllWindows()
                cv2.waitKey(1) # cannot close window on macOS without this line
                return
# visualize_keypoint_sequences(keypoints)

In [6]:
def process_sequence_features(keypoints, angles, distances):
    data_length = keypoints.shape[0]
    sequence_length = keypoints.shape[1]
    keypoints = keypoints.reshape(data_length*sequence_length, -1)
    angles = angles.reshape(data_length*sequence_length, -1)
    distances = distances.reshape(data_length*sequence_length, -1)
    features = np.concatenate((keypoints, angles, distances), -1)
    df = pd.DataFrame(features)
    np.save('gesture_learning/Fall 2020/models/gru/mean',df.mean())
    np.save('gesture_learning/Fall 2020/models/gru/std',df.std())
    df = (df-df.mean())/df.std()
    df = df.fillna(0)
    features = df.to_numpy().reshape(data_length, sequence_length, -1)
    return features
X = process_sequence_features(keypoints, angles, distances)
#normalizer = tf.keras.layers.experimental.preprocessing.Normalization()
#normalizer.adapt(X)
# X_train, X_val, y_train, y_val = train_test_split(processed_keypoints, labels, test_size=0.2, random_state=0)
print(X.shape)

(3141, 37, 483)


In [7]:
sequence_length = X.shape[1]
sequence_labels = np.tile(np.expand_dims(labels,(1,2)),[1,sequence_length,1])
print(sequence_labels.shape)

(3141, 37, 1)


## RNN Classification

In [24]:
model_lstm = tf.keras.Sequential([layers.Masking(), layers.LSTM(GESTURE_TYPES, activation=None), layers.Activation('softmax')])
model_lstm.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_lstm.fit(X, labels, epochs=20, validation_split=0.2)

Epoch 1/20
79/79 [==============================] - 3s 19ms/step - loss: 1.9801 - accuracy: 0.3759 - val_loss: 1.0546 - val_accuracy: 0.6677
Epoch 2/20
79/79 [==============================] - 1s 15ms/step - loss: 0.7404 - accuracy: 0.7849 - val_loss: 0.7906 - val_accuracy: 0.7440
Epoch 3/20
79/79 [==============================] - 1s 15ms/step - loss: 0.6055 - accuracy: 0.8274 - val_loss: 0.6894 - val_accuracy: 0.7854
Epoch 4/20
79/79 [==============================] - 1s 15ms/step - loss: 0.3791 - accuracy: 0.8829 - val_loss: 0.6299 - val_accuracy: 0.8045
Epoch 5/20
79/79 [==============================] - 1s 16ms/step - loss: 0.3960 - accuracy: 0.8932 - val_loss: 0.7045 - val_accuracy: 0.7997
Epoch 6/20
79/79 [==============================] - 1s 15ms/step - loss: 0.3216 - accuracy: 0.9025 - val_loss: 0.6226 - val_accuracy: 0.8029
Epoch 7/20
79/79 [==============================] - 1s 14ms/step - loss: 0.2993 - accuracy: 0.9112 - val_loss: 0.5583 - val_accuracy: 0.8299
Epoch 8/20
79

In [25]:
model_gru = tf.keras.Sequential([layers.Masking(), layers.GRU(GESTURE_TYPES, activation=None), layers.Activation('softmax')])
model_gru.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_gru.fit(X, labels, epochs=20, validation_split=0.2)

Epoch 1/20
79/79 [==============================] - 2s 16ms/step - loss: 1.8634 - accuracy: 0.3790 - val_loss: 0.8074 - val_accuracy: 0.7679
Epoch 2/20
79/79 [==============================] - 1s 13ms/step - loss: 0.5970 - accuracy: 0.8112 - val_loss: 0.6160 - val_accuracy: 0.8060
Epoch 3/20
79/79 [==============================] - 1s 13ms/step - loss: 0.4184 - accuracy: 0.8723 - val_loss: 0.5625 - val_accuracy: 0.8235
Epoch 4/20
79/79 [==============================] - 1s 13ms/step - loss: 0.3426 - accuracy: 0.8962 - val_loss: 0.5459 - val_accuracy: 0.8331
Epoch 5/20
79/79 [==============================] - 1s 13ms/step - loss: 0.2990 - accuracy: 0.9035 - val_loss: 0.5202 - val_accuracy: 0.8490
Epoch 6/20
79/79 [==============================] - 1s 13ms/step - loss: 0.2639 - accuracy: 0.9185 - val_loss: 0.5073 - val_accuracy: 0.8521
Epoch 7/20
79/79 [==============================] - 1s 13ms/step - loss: 0.2406 - accuracy: 0.9272 - val_loss: 0.5040 - val_accuracy: 0.8458
Epoch 8/20
79

In [22]:
model_gru_stateful = tf.keras.Sequential([layers.Masking(batch_input_shape=(1,37,483)), layers.GRU(GESTURE_TYPES, activation=None, stateful=True), layers.Activation('softmax')])
model_gru_stateful.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_gru_stateful.fit(X, labels, batch_size=1, epochs=20, validation_split=0.2)

Epoch 1/20
2512/2512 [==============================] - 18s 7ms/step - loss: 1.4429 - accuracy: 0.5611 - val_loss: 1.2721 - val_accuracy: 0.6948
Epoch 2/20
2512/2512 [==============================] - 18s 7ms/step - loss: 1.0300 - accuracy: 0.7769 - val_loss: 1.3480 - val_accuracy: 0.7345
Epoch 3/20
2512/2512 [==============================] - 17s 7ms/step - loss: 0.9433 - accuracy: 0.8069 - val_loss: 1.8012 - val_accuracy: 0.6916
Epoch 4/20
2512/2512 [==============================] - 18s 7ms/step - loss: 1.0155 - accuracy: 0.8145 - val_loss: 1.3034 - val_accuracy: 0.7599
Epoch 5/20
2512/2512 [==============================] - 18s 7ms/step - loss: 0.9008 - accuracy: 0.8335 - val_loss: 1.3439 - val_accuracy: 0.7679
Epoch 6/20
2512/2512 [==============================] - 17s 7ms/step - loss: 0.9291 - accuracy: 0.8293 - val_loss: 1.4563 - val_accuracy: 0.7536
Epoch 7/20
2512/2512 [==============================] - 17s 7ms/step - loss: 0.8980 - accuracy: 0.8416 - val_loss: 1.7824 - val_ac

In [24]:
model_gru_stateful.save_weights('gesture_learning/Fall 2020/models/gru_stateful/model')

GRU cells work just as well as LSTM cells (or even better), and is cheaper in theory.

In [9]:
model_bilstm = tf.keras.Sequential([layers.Masking(), tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(GESTURE_TYPES, activation=None)), tf.keras.layers.Activation('softmax')])
model_bilstm.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_bilstm.fit(X, labels, epochs=20, validation_split=0.2)

Train on 2512 samples, validate on 629 samples
Epoch 1/20
2512/2512 [==============================] - 25s 10ms/sample - loss: 2.1998 - accuracy: 0.3790 - val_loss: 1.3792 - val_accuracy: 0.5835
Epoch 2/20
2512/2512 [==============================] - 17s 7ms/sample - loss: 0.9320 - accuracy: 0.7480 - val_loss: 0.9101 - val_accuracy: 0.7615
Epoch 3/20
2512/2512 [==============================] - 17s 7ms/sample - loss: 0.6225 - accuracy: 0.8312 - val_loss: 0.8323 - val_accuracy: 0.7822
Epoch 4/20
2512/2512 [==============================] - 17s 7ms/sample - loss: 0.4682 - accuracy: 0.8682 - val_loss: 0.7740 - val_accuracy: 0.8029
Epoch 5/20
2512/2512 [==============================] - 16s 6ms/sample - loss: 0.6272 - accuracy: 0.8439 - val_loss: 0.8068 - val_accuracy: 0.7679
Epoch 6/20
2512/2512 [==============================] - 11s 4ms/sample - loss: 0.4971 - accuracy: 0.8686 - val_loss: 0.8009 - val_accuracy: 0.8013
Epoch 7/20
2512/2512 [==============================] - 10s 4ms/sample

Making LSTM bidirectional harms performance in long sequence case.

In [10]:
model_rnn = tf.keras.Sequential([layers.Masking() ,tf.keras.layers.SimpleRNN(GESTURE_TYPES, activation=None), tf.keras.layers.Activation('softmax')])
model_rnn.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_rnn.fit(X, labels, epochs=20, validation_split=0.2)

Train on 2512 samples, validate on 629 samples
Epoch 1/20
2512/2512 [==============================] - 5s 2ms/sample - loss: 3.3796 - accuracy: 0.3428 - val_loss: 2.0242 - val_accuracy: 0.4197
Epoch 2/20
2512/2512 [==============================] - 3s 1ms/sample - loss: 1.5106 - accuracy: 0.5092 - val_loss: 1.5615 - val_accuracy: 0.5167
Epoch 3/20
2512/2512 [==============================] - 3s 1ms/sample - loss: 1.2023 - accuracy: 0.6135 - val_loss: 1.3567 - val_accuracy: 0.5644
Epoch 4/20
2512/2512 [==============================] - 4s 1ms/sample - loss: 1.0113 - accuracy: 0.6708 - val_loss: 1.1681 - val_accuracy: 0.6216
Epoch 5/20
2512/2512 [==============================] - 4s 1ms/sample - loss: 0.8576 - accuracy: 0.7217 - val_loss: 1.0450 - val_accuracy: 0.6439
Epoch 6/20
2512/2512 [==============================] - 4s 2ms/sample - loss: 0.7710 - accuracy: 0.7508 - val_loss: 0.9199 - val_accuracy: 0.7011
Epoch 7/20
2512/2512 [==============================] - 4s 1ms/sample - loss:

Simple RNN gives a slightly worse performance.

In [11]:
model_lstm4 = tf.keras.Sequential([layers.Masking() ,tf.keras.layers.LSTM(128), layers.Dense(GESTURE_TYPES), layers.Activation('softmax')])
model_lstm4.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_lstm4.fit(X, labels, epochs=20, validation_split=0.2)

Train on 2512 samples, validate on 629 samples
Epoch 1/20
2512/2512 [==============================] - 16s 6ms/sample - loss: 0.7514 - accuracy: 0.7894 - val_loss: 0.5478 - val_accuracy: 0.8219
Epoch 2/20
2512/2512 [==============================] - 10s 4ms/sample - loss: 0.3271 - accuracy: 0.9001 - val_loss: 0.4696 - val_accuracy: 0.8537
Epoch 3/20
2512/2512 [==============================] - 11s 4ms/sample - loss: 0.2448 - accuracy: 0.9295 - val_loss: 0.4966 - val_accuracy: 0.8458
Epoch 4/20
2512/2512 [==============================] - 11s 4ms/sample - loss: 0.1860 - accuracy: 0.9471 - val_loss: 0.4520 - val_accuracy: 0.8553
Epoch 5/20
2512/2512 [==============================] - 11s 4ms/sample - loss: 0.1548 - accuracy: 0.9562 - val_loss: 0.4550 - val_accuracy: 0.8569
Epoch 6/20
2512/2512 [==============================] - 11s 4ms/sample - loss: 0.1132 - accuracy: 0.9717 - val_loss: 0.4368 - val_accuracy: 0.8490
Epoch 7/20
2512/2512 [==============================] - 11s 4ms/sample 

Increasing depth may improve performance.
## Sqeuence to Sequence

In [36]:

model_lstm_sequence = tf.keras.Sequential([layers.Masking(), layers.LSTM(128, return_sequences=True), layers.TimeDistributed(layers.Dense(GESTURE_TYPES, activation='softmax'))])
model_lstm_sequence.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_lstm_sequence.fit(X, sequence_labels, epochs=20, validation_split=0.2)

Train on 2512 samples, validate on 629 samples
Epoch 1/20
2512/2512 [==============================] - 18s 7ms/sample - loss: 0.2814 - accuracy: 0.6979 - val_loss: 0.2238 - val_accuracy: 0.7336
Epoch 2/20
2512/2512 [==============================] - 11s 4ms/sample - loss: 0.1840 - accuracy: 0.7912 - val_loss: 0.2065 - val_accuracy: 0.7493
Epoch 3/20
2512/2512 [==============================] - 11s 4ms/sample - loss: 0.1621 - accuracy: 0.8151 - val_loss: 0.1969 - val_accuracy: 0.7531
Epoch 4/20
2512/2512 [==============================] - 11s 5ms/sample - loss: 0.1466 - accuracy: 0.8305 - val_loss: 0.1967 - val_accuracy: 0.7575
Epoch 5/20
2512/2512 [==============================] - 12s 5ms/sample - loss: 0.1371 - accuracy: 0.8430 - val_loss: 0.1972 - val_accuracy: 0.7595
Epoch 6/20
2512/2512 [==============================] - 12s 5ms/sample - loss: 0.1287 - accuracy: 0.8512 - val_loss: 0.1955 - val_accuracy: 0.7600
Epoch 7/20
2512/2512 [==============================] - 12s 5ms/sample 

In [11]:
model_gru_sequence = tf.keras.Sequential([layers.Masking(), layers.GRU(128, return_sequences=True), layers.TimeDistributed(layers.Dense(GESTURE_TYPES, activation='softmax'))])
model_gru_sequence.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_gru_sequence.fit(X, sequence_labels, epochs=20, validation_split=0.2)

Train on 2512 samples, validate on 629 samples
Epoch 1/20
2512/2512 [==============================] - 29s 11ms/sample - loss: 0.2757 - accuracy: 0.6881 - val_loss: 0.2241 - val_accuracy: 0.7308
Epoch 2/20
2512/2512 [==============================] - 16s 6ms/sample - loss: 0.1811 - accuracy: 0.7908 - val_loss: 0.2105 - val_accuracy: 0.7432
Epoch 3/20
2512/2512 [==============================] - 16s 6ms/sample - loss: 0.1591 - accuracy: 0.8154 - val_loss: 0.1974 - val_accuracy: 0.7535
Epoch 4/20
2512/2512 [==============================] - 16s 6ms/sample - loss: 0.1444 - accuracy: 0.8321 - val_loss: 0.1990 - val_accuracy: 0.7471
Epoch 5/20
2512/2512 [==============================] - 16s 6ms/sample - loss: 0.1363 - accuracy: 0.8412 - val_loss: 0.1947 - val_accuracy: 0.7597
Epoch 6/20
2512/2512 [==============================] - 15s 6ms/sample - loss: 0.1219 - accuracy: 0.8596 - val_loss: 0.1951 - val_accuracy: 0.7634
Epoch 7/20
2512/2512 [==============================] - 16s 6ms/sample

CNN + LSTM

In [21]:
model_cnnlstm2 = tf.keras.Sequential([layers.Masking(), layers.Conv1D(20,3,activation='relu'), layers.AveragePooling1D(), layers.Conv1D(20,3), layers.LSTM(128), layers.Dense(GESTURE_TYPES, activation='softmax')])
model_cnnlstm2.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_cnnlstm2.fit(X, labels, epochs=20, validation_split=0.2)

Train on 2512 samples, validate on 629 samples
Epoch 1/20
2512/2512 [==============================] - 9s 4ms/sample - loss: 1.2806 - accuracy: 0.5641 - val_loss: 0.7703 - val_accuracy: 0.7552
Epoch 2/20
2512/2512 [==============================] - 6s 2ms/sample - loss: 0.5473 - accuracy: 0.8288 - val_loss: 0.6470 - val_accuracy: 0.8108
Epoch 3/20
2512/2512 [==============================] - 5s 2ms/sample - loss: 0.4225 - accuracy: 0.8666 - val_loss: 0.6022 - val_accuracy: 0.8140
Epoch 4/20
2512/2512 [==============================] - 6s 2ms/sample - loss: 0.4045 - accuracy: 0.8742 - val_loss: 0.5405 - val_accuracy: 0.8378
Epoch 5/20
2512/2512 [==============================] - 6s 2ms/sample - loss: 0.3325 - accuracy: 0.8993 - val_loss: 0.6007 - val_accuracy: 0.8203
Epoch 6/20
2512/2512 [==============================] - 6s 2ms/sample - loss: 0.3238 - accuracy: 0.9005 - val_loss: 0.5627 - val_accuracy: 0.8362
Epoch 7/20
2512/2512 [==============================] - 6s 2ms/sample - loss: